In [107]:
import sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler 
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

from statsmodels.tsa.deterministic import DeterministicProcess

print('hello world')

hello world


In [ ]:
df_origin = pd.read_csv('open/train.csv')

train_df = pd.read_csv('open/train.csv')
test_df = pd.read_csv('open/test.csv')
sub_df = pd.read_csv('open/sample_submission.csv')

print('Number of Data for training : {col}\n Number of Variables : {row}'
      .format(col = train_df.shape[0], row = train_df.shape[1]))

주말 / 평일 구분 (Column : days_of_week)

In [ ]:
train_df.loc[[611,612,613,614,615]] # 2019-09-07 자료가 없어...

In [ ]:
# 2018.01.01 ~ 2020.12.31 (NaN of 2019.09.07 sat ;;;;;)
# 
new_row = pd.DataFrame(train_df.loc[[613]], columns = train_df.columns)

train_df = pd.concat([train_df.iloc[:613], new_row, train_df.iloc[613:]], ignore_index = True)

train_df.loc[614,'date'] = '2019-09-07' 


print(train_df.loc[[612,613,614,615]])
len(train_df)

In [ ]:
# To distinguish weekend and weekday

# The name of new index is "days_of_week" 
# [0:Mon ~ Fri, 1:Sat, 1:Sun]

df_week = [] # List for df column "days of week"

L1 = train_df.shape[0] // 7  # 1096 // 7
L2 = train_df.shape[0] % 7  # 1096 // 7

#week = [0, 1, 2, 3, 4, 5, 6]
week = [0, 0, 0, 0, 0, 1, 1] # weekday / weekend

for i in range(0,L1,1):
    df_week.extend(week)

for i in range(0,L2,1):
    df_week.append(0)

print(len(df_week))


# test_df
t_df_week = []

t_L1 = test_df.shape[0] // 7
t_L2 = test_df.shape[0] % 7

t_week = [0, 1, 1, 0, 0, 0, 0]

for i in range(0, t_L1, 1):
    t_df_week.extend(week)

for i in range(0, t_L2, 1):
    t_df_week.append(0)

In [ ]:
# Appended New Col "days of week"
train_df['days_of_week'] = df_week

test_df['days_of_week'] = t_df_week

코드공유에서 가져온 Year / Month / Day 구분

In [ ]:
year = []
month = []
day = [] 

for date in train_df["date"] :
    y = date.split('-')[0]
    m = date.split('-')[1]
    d = date.split('-')[2]
    
    year.append(y)
    month.append(m)
    day.append(d)

train_df["year"] = year 
train_df["month"] = month 
train_df["day"] = day

year = []
month = []
day = [] 

for date in test_df["date"] :
    y = date.split('-')[0]
    m = date.split('-')[1]
    d = date.split('-')[2]
    
    year.append(y)
    month.append(m)
    day.append(d)

test_df["year"] = year 
test_df["month"] = month 
test_df["day"] = day

In [ ]:
train_df

In [ ]:
test_df

Sunshine_rate을 이용하여 Sunshine_sum의 NaN을 선형 회귀로 구하기

In [ ]:
nan_list_ss = train_df['sunshine_sum'][train_df['sunshine_sum'].isnull()].index
print(nan_list_ss)

t_nan_list_ss = test_df['sunshine_sum'][test_df['sunshine_sum'].isnull()].index
print(t_nan_list_ss)

In [ ]:
train_df['sunshine_rate'].loc[nan_list_ss].values.reshape(-1,1) 
# 사실 train 데이터에서는 sunshine_sum이 Nan인 날은 전부 sunshine_rate가 0이라 걍 0으로 채워도 될거같음
# 그리고 test 데이터는 sunshine_sum Nan이 하나밖에없어서 큰 영향을 안줄듯
# 근데 이미 다 작성해놔서 걍 안지우고 넣어둠 ㅎㅎ..

In [ ]:
print(train_df[['sunshine_rate', 'sunshine_sum']].corr(method='pearson')) # Pearson Correlation

a = train_df['sunshine_sum'].dropna()
b = train_df['sunshine_rate'].drop(nan_list_ss)
a.isnull().sum()
b.isnull().sum()

t_a = test_df['sunshine_sum'].dropna()
t_b = test_df['sunshine_rate'].drop(t_nan_list_ss)
t_a.isnull().sum()
t_b.isnull().sum()

plt.plot(a,b, 'o')
plt.plot(t_a,t_b, 'o')

In [ ]:
a = a.values.reshape(-1,1) # a is sunshine_sum
b = b.values.reshape(-1,1) # b is sunshine_rate

t_a = t_a.values.reshape(-1, 1)
t_b = t_b.values.reshape(-1, 1)

In [ ]:
line_fitter = LinearRegression() # 선형회귀 이용하여 SS의 Nan 값 예측

line_fitter.fit(b, a) # predict a(sum) through b(rate) 
pred_ss = line_fitter.predict(train_df['sunshine_rate'].loc[nan_list_ss].values.reshape(-1, 1))
predd_ss = list(pred_ss)

t_pred_ss = line_fitter.predict(test_df['sunshine_rate'].loc[t_nan_list_ss].values.reshape(-1, 1))
t_pred_ss = list(t_pred_ss)

In [ ]:
train_df["PM10"] = train_df["PM10"].fillna(train_df["PM10"].mean())
train_df["PM2.5"] = train_df["PM2.5"].fillna(train_df["PM2.5"].mean())
train_df["sunshine_sum"] = train_df["sunshine_sum"].fillna(0) # NaN값은  = 2  << 이런식으로 안들어가서
# 일단 0으로 다 채운다음에 선형회귀로 구한 값 입력

for i, j in zip(nan_list_ss, range(0,len(nan_list_ss),1)):
    train_df['sunshine_sum'].loc[i] = float(pred_ss[j])

for ii, jj in zip(t_nan_list_ss, range(0, len(t_nan_list_ss), 1)):
    test_df['sunshine_sum'].loc[ii] = float(t_pred_ss[jj])


In [ ]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())

In [ ]:
print(train_df['sunshine_sum'].loc[nan_list_ss])
print(test_df['sunshine_sum'].loc[t_nan_list_ss])

# 다 채움 근데 0으로 하는게 더 나을거같은 생각이 계속 듬

Predict Precipitation Using Linear Regression

In [ ]:
nan_list_pp= train_df['precipitation'][train_df['precipitation'].isnull()].index
print(nan_list_pp)

pp = train_df['precipitation'].dropna()
srss = train_df[['sunshine_rate','sunshine_sum','month']].drop(nan_list_pp)

t_nan_list_pp = test_df['precipitation'][train_df['precipitation'].isnull()].index
print(t_nan_list_pp)

t_pp = test_df['precipitation'].dropna()
t_srss = test_df[['sunshine_rate','sunshine_sum','month']].drop(t_nan_list_pp)

In [ ]:
scaler = MinMaxScaler()

scaler.fit(srss)

In [ ]:
scaled_srss = scaler.transform(srss)

In [ ]:
line_fitter.fit(scaled_srss, pp)

In [ ]:
#pp_sol_Nan = line_fitter.predict(scaler.transform(new_df[['sunshine_rate','sunshine_sum','month']].loc[PP_list_Nan].values))
pred_pp = line_fitter.predict(scaler.transform(train_df[['sunshine_rate','sunshine_sum','month']].loc[nan_list_pp].values))
pred_pp = list(pred_pp)


t_pred_pp = list(line_fitter.predict(scaler.transform(test_df[['sunshine_rate','sunshine_sum','month']].loc[t_nan_list_pp].values)))


# very very weird
# Minus precipitation? >> Evaporation?

print(pred_pp)
print(t_pred_pp)

print(type(t_pred_pp))
len(t_pred_pp)

In [30]:
train_df['precipitation'] = train_df['precipitation'].fillna(0)

for i, j in zip(nan_list_pp, range(0,len(pred_pp),1)):
    train_df['precipitation'].loc[i] = float(pred_pp[j])



test_df['precipitation'] = test_df['precipitation'].fillna(0)


for ii, jj in zip(t_nan_list_pp, range(0, len(t_pred_pp), 1)):
    test_df['precipitation'].loc[ii] = float(t_pred_pp[jj])

C:\Users\Admin\AppData\Local\Temp\ipykernel_18512\3578690188.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['precipitation'].loc[ii] = float(t_pred_pp[jj])


In [31]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())

date             0
precipitation    0
temp_mean        0
temp_highest     0
temp_lowest      0
PM10             0
PM2.5            0
humidity         0
sunshine_sum     0
sunshine_rate    0
wind_mean        0
wind_max         0
rental           0
days_of_week     0
year             0
month            0
day              0
dtype: int64
date             0
precipitation    0
temp_mean        0
temp_highest     0
temp_lowest      0
PM10             0
PM2.5            0
humidity         0
sunshine_sum     0
sunshine_rate    0
wind_mean        0
wind_max         0
days_of_week     0
year             0
month            0
day              0
dtype: int64


# 2018 ~ 2019, 2019 ~ 2020년도 각 월별 증가량 추이

In [105]:
train_2018 = train_df[train_df['year'] == '2018']
train_2019 = train_df[train_df['year'] == '2019']
train_2020 = train_df[train_df['year'] == '2020']



print(train_2018[train_2018['month'] == '01' ]['rental'].values.sum())
print(train_2019[train_2019['month'] == '01' ]['rental'].values.sum())
print(train_2020[train_2020['month'] == '01' ]['rental'].values.sum())
print('\n')
print(train_2018[train_2018['month'] == '02' ]['rental'].values.sum())
print(train_2019[train_2019['month'] == '02' ]['rental'].values.sum())
print(train_2020[train_2020['month'] == '02' ]['rental'].values.sum())
print('\n')
print(train_2018[train_2018['month'] == '03' ]['rental'].values.sum())
print(train_2019[train_2019['month'] == '03' ]['rental'].values.sum())
print(train_2020[train_2020['month'] == '03' ]['rental'].values.sum())

l = LinearRegression()
l.fit()

164367
495573
794162


168741
471543
762274


462661
904819
1533535
2.0150395152311598
0.6025126469763284


In [33]:
def NMAE(true, pred):
    score = np.mean(np.abs(true-pred) / true)
    return score

In [34]:
# x = np.array(train_df.drop(['date','rental'], axis = 1))
# y = train_df['rental']

# print(type(len(x)))

# s = len(x) // 3
# print(s)

# train_index = list(range(0, len(x)-s, 1))
# test_index = list(range(len(x)-s, len(x), 1))

# print(len(train_index))
# print(len(test_index))

# # x_train, x_test = x[]
# # y_train, y_test = 

In [35]:
# acc_history1 = []
# acc_history2 = []
# acc_history3 = []


# model1 = RandomForestRegressor()
# model1.fit(x_train, y_train)

# y1_pred = model1.predict(x_test)
# acc_history1.append(NMAE(y_test, y1_pred))



# model2 = GradientBoostingRegressor()
# model2.fit(x_train, y_train)

# y2_pred = model2.predict(x_test)
# acc_history2.append(NMAE(y_test, y2_pred))



# model3 = LinearRegression()
# model3.fit(x_train, y_train)

# y3_pred = model3.predict(x_test)
# acc_history3.append(NMAE(y_test, y3_pred))

# print(np.mean(acc_history1), np.mean(acc_history2), np.mean(acc_history3))

In [36]:

x = np.array(train_df.drop(['date','rental'], axis =1))
y = train_df['rental']

kf = KFold(n_splits = 15, shuffle = False)

In [37]:
acc_history1 = []
acc_history2 = []


for train_index, test_index in kf.split(x):

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]


    model1 = RandomForestRegressor()
    model1.fit(x_train, y_train)

    y1_pred = model1.predict(x_test)
    acc_history1.append(NMAE(y_test, y1_pred))


    model2 = GradientBoostingRegressor()
    model2.fit(x_train, y_train)

    y2_pred = model2.predict(x_test)
    acc_history2.append(NMAE(y_test, y2_pred))

print(np.mean(acc_history1), np.mean(acc_history2))

KeyboardInterrupt: 

In [ ]:
test_data =test_df.drop(['date'], axis =1)


test_pred1 = model1.predict(test_data)

test_pred2 = model2.predict(test_data)

In [ ]:
print(test_pred1)

In [ ]:
print(test_pred2)

In [ ]:
for i in range(0, len(test_pred2), 1) :
    sub_df['rental'].loc[i] = test_pred2[i]

sub_df.to_csv('/content/drive/MyDrive/2022DACON-bike-Hyeong/2022DACON-bike-Hyeong/open/sample_submission2.csv', index=False)


In [ ]:
fig1 = plt.plot(test_pred2)

fig2 = plt.plot(train_df['rental'])

In [ ]:
def NMAE(true, pred):
    score = np.mean(np.abs(true-pred) / true)
    return score

In [ ]:
nmae_gbr = []
nmae_rf = []
nmae_lr = []

for i in range(0, 10, 1):

    train_x, val_x, train_y, val_y = train_test_split(x, y, test_size = 0.25, shuffle=True)

    reg = GradientBoostingRegressor()
    reg.fit(train_x, train_y)
    val_pred = reg.predict(val_x)
    nmae_gbr.append(NMAE(val_y, val_pred))


    regRF = RandomForestRegressor()
    regRF.fit(train_x, train_y)
    val_pred = regRF.predict(val_x)
    nmae_rf.append(NMAE(val_y, val_pred))
    
    regLR = LinearRegression()
    regLR.fit(train_x, train_y)
    val_pred = regLR.predict(val_x)
    nmae_lr.append(NMAE(val_y, val_pred))


In [ ]:
plt.plot(range(0,10,1), nmae_gbr, label = 'GBR_precip')
plt.plot(range(0,10,1), nmae_rf, label = 'RF_precip')
plt.plot(range(0,10,1), nmae_lr, label = 'LR_precip' )
plt.legend()

gbr_precip = sum(nmae_gbr)/len(nmae_gbr)
rf_precip = sum(nmae_rf)/len(nmae_gbr)
lr_precip = sum(nmae_lr)/len(nmae_lr)

print(gbr_precip)
print(rf_precip)
print(lr_precip)

In [ ]:
x1 = train_df.drop(['precipitation','date','rental'], axis=1)
y1 = train_df['rental']

In [ ]:
nmae_gbr = []
nmae_rf = []
nmae_lr = []

for i in range(0, 10, 1):
   
    train_x, val_x, train_y, val_y = train_test_split(x, y, test_size = 0.25, shuffle=True)

    reg = GradientBoostingRegressor()
    reg.fit(train_x, train_y)
    val_pred = reg.predict(val_x)
    nmae_gbr.append(NMAE(val_y, val_pred))


    regRF = RandomForestRegressor()
    regRF.fit(train_x, train_y)
    val_pred = regRF.predict(val_x)
    nmae_rf.append(NMAE(val_y, val_pred))

    regLR = LinearRegression()
    regLR.fit(train_x, train_y)
    val_pred = regLR.predict(val_x)
    nmae_lr.append(NMAE(val_y, val_pred))
    

In [ ]:
plt.plot(range(0,10,1), nmae_gbr, label = 'GBR_non_precip')
plt.plot(range(0,10,1), nmae_rf, label = 'RF_non_precip')
plt.plot(range(0,10,1), nmae_lr, label = 'LR_non_precip' )
plt.legend()

gbr_non_precip = sum(nmae_gbr)/len(nmae_gbr)
rf_non_precip = sum(nmae_rf)/len(nmae_gbr)
lr_non_precip = sum(nmae_lr)/len(nmae_lr)

print(gbr_non_precip)
print(rf_non_precip)
print(lr_non_precip)

In [ ]:
print(gbr_non_precip)
print(rf_non_precip)

In [ ]:
print(gbr_precip)
print(rf_precip)